In [1]:
import os

In [2]:
use_gpu = True

if use_gpu:
    import cudf as pd
else:
    import pandas as pd

In [3]:
preprocess_dir = 'storage/preprocessing/220311_baseline/'

In [4]:
from utils import save_csv

****Input dataset****

In [5]:
%%time
transactions = pd.read_csv('storage/transactions_train.csv')
articles = pd.read_csv('storage/articles.csv')
customers = pd.read_csv('storage/customers.csv')

CPU times: user 1.25 s, sys: 1.34 s, total: 2.59 s
Wall time: 2.63 s


In [6]:
%%time
from utils import train_test_split
trn_transactions,val_transactions = train_test_split(transactions,gpu=use_gpu)
del(transactions)

CPU times: user 17.7 ms, sys: 33.6 ms, total: 51.4 ms
Wall time: 50.2 ms


In [6]:
#trn_transactions = trn_transactions[trn_transactions['t_dat'] > pd.to_datetime('2020-08-01')]

****Feature engineering with past purchase history****

In [7]:
def past_purchase_count_vector(df,art_df,selected_feature,postfix='_countvec'):
    df = df.merge(art_df[['article_id',selected_feature]],on='article_id')
    df['count'] = 1
    norm = df.groupby(['customer_id'])['count'].count().reset_index()
    norm.rename(columns={'count':'norm'},inplace=True)
    count = df.groupby(['customer_id',selected_feature])['count'].count().reset_index()
    count = count.merge(norm,on='customer_id')
    count['count'] = count['count'] / count['norm']
    count = count.rename(columns={'count':selected_feature+postfix})
    del(norm)
    return count[['customer_id',selected_feature,selected_feature+postfix]]

In [8]:
%%time
selected_features = [
    'product_group_name', 'product_type_name', 
    'graphical_appearance_name', 'perceived_colour_value_name', 'colour_group_code', 
    'index_name', 'index_group_name', 
    'section_name', 'department_name',
]
for selected_feature in selected_features:
    count = past_purchase_count_vector(trn_transactions[['customer_id','article_id']],articles,selected_feature)
    save_csv(count,preprocess_dir,selected_feature+'_countvec.csv')

CPU times: user 2.04 s, sys: 5.5 s, total: 7.54 s
Wall time: 8.67 s


****Feature engineering on repeated purchase****

In [9]:
%%time
trn_transactions['count'] = 1
count = trn_transactions[['customer_id','article_id','count']].groupby(['customer_id','article_id'])['count'].count().reset_index()
norm = trn_transactions[['customer_id','article_id']].groupby('customer_id').count().reset_index().rename(columns={'article_id':'norm'})
count = count.merge(norm,on='customer_id')
count['count'] = count['count'] / count['norm']
count.drop(columns=['norm'],inplace=True)

CPU times: user 93.2 ms, sys: 159 ms, total: 252 ms
Wall time: 251 ms


In [ ]:
save_csv(count.to_pandas(),preprocess_dir,'repeated_purchase_prob.csv')